In [79]:
import pandas as pd

In [80]:
data = pd.read_csv('./abcnews-date-text.csv',error_bad_lines=False)

C:\Program Files\KMSpico\temp\ipykernel_10248\338514806.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('./abcnews-date-text.csv',error_bad_lines=False)


In [81]:
data.columns

Index(['publish_date', 'headline_text'], dtype='object')

In [82]:
data['headline_text'].head(5)

0    aba decides against community broadcasting lic...
1       act fire witnesses must be aware of defamation
2       a g calls for infrastructure protection summit
3             air nz staff in aust strike for pay rise
4        air nz strike to affect australian travellers
Name: headline_text, dtype: object

In [83]:
data_text = data[['headline_text']] # makes it a dataframe

In [84]:
type(data_text)

pandas.core.frame.DataFrame

In [85]:
data_text

,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers
...,...
1244179,two aged care residents die as state records 2...
1244180,victoria records 5;919 new cases and seven deaths
1244181,wa delays adopting new close contact definition
1244182,western ringtail possums found badly dehydrate...


In [86]:
data_text.index

RangeIndex(start=0, stop=1244184, step=1)

In [87]:
data_text['index'] = data_text.index

In [88]:
data_text

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4
...,...,...
1244179,two aged care residents die as state records 2...,1244179
1244180,victoria records 5;919 new cases and seven deaths,1244180
1244181,wa delays adopting new close contact definition,1244181
1244182,western ringtail possums found badly dehydrate...,1244182


In [89]:
documents = data_text

### Data Preprocessing

In [90]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np 
np.random.seed(42)

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [91]:
def lemmatize_stemming(text):
    p = PorterStemmer()
    return p.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

In [92]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and \
            len(token)>3:
            result.append(lemmatize_stemming(token))
    return result

Lets see it works or not in one sample

In [93]:
doc_sample = documents[documents['index']==4310].values[0][0]

In [94]:
doc_sample

'ratepayers group wants compulsory local govt voting'

In [95]:
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)

print(words)

original document: 
['ratepayers', 'group', 'wants', 'compulsory', 'local', 'govt', 'voting']


In [96]:
# after preprocessing
preprocess(doc_sample)

['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']

Lets now preprocess the whole documents

In [97]:
processed_docs = documents['headline_text'].map(preprocess)

#### Bag of Words on the Data Set

In [98]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [99]:
len(dictionary)

70293

In [100]:
count = 0
for k,v in dictionary.iteritems():
    print(k,v)
    count+=1
    if count >10:
        break 

0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


Filter out tokens in the dictionary by their frequency.

In [101]:
dictionary.filter_extremes(no_below=15,no_above=0.5,keep_n=100000)

#### Gensim doc2bow

In [102]:
processed_docs

0                         [decid, commun, broadcast, licenc]
1                                         [wit, awar, defam]
2                     [call, infrastructur, protect, summit]
3                                [staff, aust, strike, rise]
4                       [strike, affect, australian, travel]
                                 ...                        
1244179                    [age, care, resid, state, record]
1244180               [victoria, record, case, seven, death]
1244181              [delay, adopt, close, contact, definit]
1244182    [western, ringtail, possum, badli, dehydr, hea...
1244183                  [make, close, covid, contact, rule]
Name: headline_text, Length: 1244184, dtype: object

In [103]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [104]:
bow_corpus[4310]

[(162, 1), (240, 1), (292, 1), (589, 1), (839, 1), (3579, 1), (3580, 1)]

#### TF-IDF

In [105]:
from gensim import corpora,models

In [106]:
tfidf = models.TfidfModel(bow_corpus) #fit the model

In [107]:
type(tfidf)

gensim.models.tfidfmodel.TfidfModel

In [108]:
corpus_tfidf = tfidf[bow_corpus] # apply model to the corpus

In [109]:
corpus_tfidf[4]

[(14, 0.47235494692269364),
 (15, 0.5715156253616932),
 (16, 0.38223210042906264),
 (17, 0.5514973395100641)]

#### Running LDA using Bag of Words

In [110]:
lda_model = gensim.models.LdaMulticore(
    bow_corpus,
    num_topics = 10,
    id2word = dictionary,
    passes = 2,
)

Lets visualize our topics

In [111]:
for idx,topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.015*"rural" + 0.015*"news" + 0.015*"farmer" + 0.013*"price" + 0.010*"nation" + 0.010*"busi" + 0.009*"labor" + 0.007*"rise" + 0.007*"liber" + 0.007*"drive"
Topic: 1 
Words: 0.014*"live" + 0.014*"chang" + 0.011*"centr" + 0.011*"group" + 0.011*"australia" + 0.010*"coronaviru" + 0.010*"work" + 0.010*"forc" + 0.009*"covid" + 0.008*"commun"
Topic: 2 
Words: 0.018*"world" + 0.015*"south" + 0.015*"australia" + 0.011*"market" + 0.010*"australian" + 0.009*"die" + 0.009*"donald" + 0.009*"lead" + 0.009*"hold" + 0.008*"record"
Topic: 3 
Words: 0.022*"plan" + 0.022*"council" + 0.015*"govern" + 0.010*"road" + 0.009*"elect" + 0.009*"indigen" + 0.008*"fund" + 0.007*"feder" + 0.007*"tasmania" + 0.006*"reveal"
Topic: 4 
Words: 0.022*"health" + 0.018*"hospit" + 0.015*"countri" + 0.015*"protest" + 0.013*"coronaviru" + 0.013*"urg" + 0.012*"flood" + 0.010*"hour" + 0.010*"covid" + 0.009*"budget"
Topic: 5 
Words: 0.032*"interview" + 0.016*"coast" + 0.015*"school" + 0.014*"driver" + 0.014*"fa

#### LDA with TF-IDF

In [113]:
lda_model_tfidf = gensim.models.LdaMulticore(
    corpus_tfidf,
    num_topics = 10,
    id2word = dictionary,
    passes = 2
)

Lets evaluate

In [ ]:
processed_docs[4310]

['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']

In [ ]:
for index,score in sorted(lda_model[bow_corpus[4310]],key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.5960084199905396	 
Topic: 0.022*"plan" + 0.021*"council" + 0.015*"govern" + 0.010*"road" + 0.009*"elect" + 0.009*"indigen" + 0.008*"fund" + 0.007*"tasmania" + 0.007*"feder" + 0.006*"reveal"

Score: 0.30389174818992615	 
Topic: 0.014*"live" + 0.014*"chang" + 0.012*"coronaviru" + 0.011*"australia" + 0.011*"centr" + 0.011*"group" + 0.010*"work" + 0.010*"forc" + 0.009*"covid" + 0.009*"commun"

Score: 0.012519883923232555	 
Topic: 0.017*"world" + 0.015*"south" + 0.015*"australia" + 0.011*"market" + 0.010*"australian" + 0.009*"record" + 0.009*"die" + 0.009*"lead" + 0.009*"donald" + 0.009*"hold"

Score: 0.012512230314314365	 
Topic: 0.015*"rural" + 0.015*"farmer" + 0.015*"news" + 0.013*"price" + 0.010*"nation" + 0.010*"busi" + 0.008*"labor" + 0.007*"rise" + 0.007*"drive" + 0.007*"liber"

Score: 0.012511689215898514	 
Topic: 0.022*"health" + 0.018*"hospit" + 0.015*"countri" + 0.015*"protest" + 0.013*"coronaviru" + 0.013*"urg" + 0.012*"flood" + 0.010*"hour" + 0.010*"covid" + 0.009*"bu

In [ ]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'

bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(
        score, lda_model.print_topic(index, 5)))


Score: 0.3444737493991852	 Topic: 0.028*"charg" + 0.020*"trump" + 0.018*"woman" + 0.017*"murder" + 0.017*"miss"
Score: 0.21066319942474365	 Topic: 0.017*"adelaid" + 0.015*"say" + 0.014*"deal" + 0.013*"speak" + 0.011*"break"
Score: 0.1808459758758545	 Topic: 0.021*"sydney" + 0.019*"kill" + 0.017*"china" + 0.015*"trial" + 0.012*"australia"
Score: 0.16391858458518982	 Topic: 0.022*"health" + 0.018*"hospit" + 0.015*"countri" + 0.015*"protest" + 0.013*"coronaviru"
Score: 0.01668521761894226	 Topic: 0.043*"polic" + 0.018*"north" + 0.016*"crash" + 0.016*"drug" + 0.016*"concern"
Score: 0.01668384112417698	 Topic: 0.015*"rural" + 0.015*"farmer" + 0.015*"news" + 0.013*"price" + 0.010*"nation"
Score: 0.016683178022503853	 Topic: 0.014*"live" + 0.014*"chang" + 0.012*"coronaviru" + 0.011*"australia" + 0.011*"centr"
Score: 0.01668296754360199	 Topic: 0.022*"plan" + 0.021*"council" + 0.015*"govern" + 0.010*"road" + 0.009*"elect"
Score: 0.016682008281350136	 Topic: 0.032*"interview" + 0.015*"coast" + 